In [134]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
os.chdir('/home2/s439906/project/CASP16/monomer/')

In [135]:
measure = "GDT_TS"
measure = "GDT_HA"
# model = "first"
model = "best"
# mode = "easy"
# mode = "medium"
# mode = "hard"
mode = "all"
impute_value = -2

sys.path.append('/home2/s439906/project/CASP16/monomer/')
os.chdir('/home2/s439906/project/CASP16/monomer/')
heatmap_dir = "./heatmap/"
heatmap_file = f"heatmap_CASP16_{model}_{mode}_impute={impute_value}_equal_weight.csv"
heatmap_data = pd.read_csv(heatmap_dir + heatmap_file, index_col=0)
heatmap_data
raw_file = f"{measure}-{model}-{mode}-raw.csv"
score_path = "./group_by_target_EU/"
raw_data = pd.read_csv(score_path + raw_file, index_col=0)
mask = raw_data.isna()

In [ ]:
sum = heatmap_data.sum(axis=1)
sorted_indices = sum.sort_values(ascending=True).index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(
    drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)
sorted_mask = pd.DataFrame(
    mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)
# use mask to mask the data. will be used for heatmap
masked_data = sorted_heatmap_data.copy()
masked_data[sorted_mask] = np.nan


# set up the colormap
cmap = plt.cm.YlGn
cmap = ListedColormap(cmap(np.linspace(0, 1, 256)))
cmap.set_bad(color='gray')  # set the masked area to gray
# set up the figure and gridspec
fig = plt.figure(figsize=(45, 30))
gs = GridSpec(1, 2, width_ratios=[4, 1], wspace=0.1)

# # 添加行注解的颜色条
# ax_column_colors = fig.add_subplot(gs[0])
# row_colors = column_annotations.map({'Group 1': 'red', 'Group 2': 'blue', 'Group 3': 'green'})
# for idx, color in enumerate(row_colors):
#     ax_column_colors.add_patch(plt.Rectangle((0, idx), 1, 1, color=color))
# ax_column_colors.set_xlim(0, 1)
# ax_column_colors.set_ylim(len(row_colors), 0)
# ax_column_colors.axis("off")  # 隐藏坐标轴


# plot the heatmap
ax0 = fig.add_subplot(gs[0])
sns.heatmap(masked_data, cmap=cmap, cbar=True, ax=ax0)
ax0.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the row sum
ax0.set_xticklabels(sorted_heatmap_data.columns, rotation=45, ha='right')
ax0.tick_params(axis='x', labelsize=16)
ax0.tick_params(axis='y', labelsize=16)
cbar = ax0.collections[0].colorbar
cbar.ax.tick_params(labelsize=24)
ax0.set_title(
    "Heatmap for z-scores for monomers", fontsize=32)
# plot the row sum
ax1 = fig.add_subplot(gs[1], sharey=ax0)
y_pos = range(len(sorted_sum))
y_pos = [i+0.5 for i in y_pos]  # change the position of the bars
ax1.barh(y_pos, sorted_sum, color='tan')
ax1.set_yticks(range(len(sorted_sum)))
ax1.set_yticklabels(
    [f'{i}' for i in sorted_indices], rotation=0)  # use the same order as the heatmap
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)
ax1.invert_yaxis()  # flip the y-axis
ax1.set_xlabel("Sum", fontsize=16)
ax1.set_title("Group sum z-scores", fontsize=20)
# save the figure
plt.tight_layout()
plt.show()

In [138]:
sum = heatmap_data.sum(axis=1)
sorted_indices = sum.sort_values(ascending=False).index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(
    drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)
sorted_mask = pd.DataFrame(
    mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)
# use mask to mask the data. will be used for heatmap
masked_data = sorted_heatmap_data.copy()
masked_data[sorted_mask] = np.nan

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm

# Step 1: Calculate row sums
sum = heatmap_data.sum(axis=1)

# Step 2: Sort indices based on row sums
sorted_indices = sum.sort_values(ascending=False).index

# Step 3: Sort the heatmap data and row sums, reset index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)

# Step 4: Sort mask based on sorted indices
sorted_mask = pd.DataFrame(mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)

# Step 5: Mask the sorted heatmap data
masked_data = sorted_heatmap_data.where(~sorted_mask)

# Step 6: Create custom colormap
colors = ["blue", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
cmap.set_bad(color='gray')  # Set the masked area to gray

# Step 7: Create a diverging norm to set midpoint at 0
norm = TwoSlopeNorm(vmin=masked_data.min().min(), vmax=masked_data.max().max(), vcenter=0)

# Step 8: Plot heatmap
plt.figure(figsize=(30, 30))
sns.heatmap(
    masked_data,
    annot=False,                # If you want to display the values, set to True
    cmap=cmap,                  # Use the custom blue-white-red colormap
    norm=norm,                  # Use diverging norm to center at 0
    linewidths=0.2,             # Set the linewidth between cells
    linecolor='white',          # Set the line color
    cbar_kws={'label': 'Intensity'},  # Add label to the color bar
    xticklabels=masked_data.columns,  # Use original column names as labels
    yticklabels=sorted_indices,       # Use sorted indices as row labels
    square=True,                # Make each cell square
    mask=sorted_mask            # Apply mask to the heatmap
)

# Set title and labels
plt.title('Heatmap')
plt.xlabel('Groups')
plt.ylabel('Evaluation Units')

# Show heatmap
plt.show()


In [140]:
measure = "GDT_TS"
measure = "GDT_HA"
# model = "first"
model = "best"
# mode = "easy"
# mode = "medium"
# mode = "hard"
mode = "all"
impute_value = -2
sys.path.append('/home2/s439906/project/CASP16/monomer/')
os.chdir('/home2/s439906/project/CASP16/monomer/')

heatmap_dir = "./group_by_target_EU/"
heatmap_file = f"{measure}-{model}-{mode}-raw.csv"
heatmap_data = pd.read_csv(heatmap_dir + heatmap_file, index_col=0)
heatmap_data
raw_file = f"{measure}-{model}-{mode}-raw.csv"
score_path = "./group_by_target_EU/"
raw_data = pd.read_csv(score_path + raw_file, index_col=0)
mask = raw_data.isna()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm

# Step 1: Calculate row sums
sum = heatmap_data.sum(axis=1)

# Step 2: Sort indices based on row sums
sorted_indices = sum.sort_values(ascending=False).index

# Step 3: Sort the heatmap data and row sums, reset index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)

# Step 4: Sort mask based on sorted indices
sorted_mask = pd.DataFrame(mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)

# Step 5: Mask the sorted heatmap data
masked_data = sorted_heatmap_data.where(~sorted_mask)

# Step 6: Fill NaN values in masked_data with 50 for distance calculation
filled_data = masked_data.fillna(50)

# Step 7: Create custom colormap
colors = ["blue", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
cmap.set_bad(color='gray')  # Set the masked area to gray

# Step 8: Create a diverging norm to set midpoint at 50
norm = TwoSlopeNorm(vmin=filled_data.min().min(), vmax=filled_data.max().max(), vcenter=50)

# Step 9: Plot heatmap using clustermap
sns.clustermap(
    figsize=(12, 18),
    data=filled_data,
    dendrogram_ratio=(0.1, 0.1),  # Reduce the space taken by dendrograms
    cmap=cmap,                  # Use the custom blue-white-red colormap
    norm=norm,                  # Use diverging norm to center at 50
    linewidths=0.01,             # Set the linewidth between cells
    linecolor='white',          # Set the line color
    # cbar_kws={'label': 'Intensity'},  # Add label to the color bar
    xticklabels=filled_data.columns,  # Use original column names as labels
    yticklabels=sorted_indices,       # Use sorted indices as row labels
    mask=sorted_mask,            # Apply mask to the heatmap
        square=True, 
        method="ward"
)

# Show heatmap
plt.show()


In [ ]:
column_annotations={'T1201': 'human',
 'H1202': 'human',
 'H1204': 'nanobody',
 'T1207': 'viral protein',
 'M1209': 'antibody-antigen',
 'T1210': 'normal',
 'M1211': 'viral protein',
 'T1214': 'bacteria',
 'M1212': 'human transposon',
 'T1212': 'human transposon',
 'M1216': 'bacteria',
 'T1219': 'human immunity peptide',
 'T1206': 'viral protein',
 'H1208': 'bacteria',
 'M1221': 'bacteria',
 'T1208s1': 'bacteria',
 'T1208s2': 'bacteria',
 'H1215': 'nanobody',
 'M1224': 'bacteria',
 'T1226': 'viral protein',
 'H1213': 'viral protein',
 'T1249v1': 'viral protein',
 'T1249v2': 'viral protein',
 'M1228v1': 'bacteria',
 'M1228v2': 'bacteria',
 'T1228v1': 'bacteria',
 'T1228v2': 'bacteria',
 'H1217': 'viral protein',
 'T1231': 'viral protein',
 'T1218': 'bacteria',
 'H1220': 'viral protein',
 'T1220s1': 'viral protein',
 'H1222': 'antibody-antigen',
 'H1223': 'antibody-antigen',
 'M1239v1': 'bacteria',
 'M1239v2': 'bacteria',
 'T1239v1': 'bacteria',
 'T1239v2': 'bacteria',
 'H1225': 'antibody-antigen',
 'H1227': 'bacteria',
 'T1227s1': 'bacteria',
 'T1243': 'bacteria',
 'H1229': 'viral protein',
 'H1230': 'viral protein',
 'T1229s1': 'viral protein',
 'T1230s1': 'viral protein',
 'H1232': 'antibody-antigen',
 'H1233': 'antibody-antigen',
 'H1236': 'viral protein',
 'T1234': 'viral protein',
 'T1235': 'viral protein',
 'T1266': 'human protein',
 'T1237': 'bacteria',
 'T1240': 'bacteria',
 'T1269': 'human protein',
 'H1244': 'protozoan parasites',
 'T1244s1': 'protozoan parasites',
 'T1244s2': 'protozoan parasites',
 'H1245': 'fast evolving',
 'T1245s1': 'fast evolving',
 'T1245s2': 'fast evolving',
 'T1246': 'bacteria',
 'H1258': 'human',
 'T1257': 'viral protein',
 'M1271': 'protozoan parasite',
 'T1259': 'bacteria',
 'T1271s1': 'protozoan parasite',
 'T1271s2': 'protozoan parasite',
 'T1271s3': 'protozoan parasite',
 'T1271s4': 'protozoan parasite',
 'T1271s5': 'protozoan parasite',
 'T1271s6': 'protozoan parasite',
 'T1271s7': 'protozoan parasite',
 'T1271s8': 'protozoan parasite',
 'H1265': 'immunity',
 'T1274': 'bacteria',
 'H1267': 'bacteria',
 'T1267s1': 'bacteria',
 'T1267s2': 'bacteria',
 'M1268': 'bacteria',
 'T1270': 'bacteria',
 'T1278': 'bacteria',
 'T1279': 'bacteria',
 'M1282': 'viral protein',
 'T1280': 'viral protein',
 'T1284': 'viral protein',
 'H1272': 'mammal protien',
 'T1272s2': 'mammal protien',
 'T1272s3': 'mammal protien',
 'T1272s4': 'mammal protien',
 'T1272s5': 'mammal protien',
 'T1272s6': 'mammal protien',
 'T1272s7': 'mammal protien',
 'T1272s8': 'mammal protien',
 'T1272s9': 'mammal protien',
 'M1276': 'ants protein',
 'T1276': 'ants protein',
 'M1293': 'viral protein',
 'T1292': 'bacteria',
 'T1294v1': 'moulds',
 'T1294v2': 'moulds',
 'M1287': 'bacteria',
 'T1200': 'bacteria',
 'T1295': 'nanobody',
 'T1300': 'bacteria',
 'M1296': 'antibody',
 'M1297': 'human',
 'T1269v1': 'human',
 'T1298': 'bacteria',
 'T1219v1': 'human',
 'T1219v2': 'human',
 'T1299': 'Archaea'}


for i in filled_data.columns:
    print(f'"{i}":')

column_information_1 = {
"T1201": 'human protein',
"T1206-D1": 'viral protein',
"T1207-D1": 'viral protein',
"T1208s1-D1": 'bacteria protein',
"T1208s2-D1": 'bacteria protein',
"T1210": 'normal protein',
"T1212-D1": 'human protein',
"T1218-D1":  'bacteria protein',
"T1218-D2": 'bacteria protein',
"T1218-D3":'bacteria protein',
"T1220s1": 'viral protein',
"T1226-D1": 'viral protein',
"T1227s1-D1": 'bacteria protein',
"T1228-D1-all":'bacteria protein',
"T1228-D2-all":'bacteria protein',
"T1228-D3-all":'bacteria protein',
"T1228-D4-all":'bacteria protein',
"T1230s1-D1": 'viral protein',
"T1231-D1": 'viral protein',
"T1234-D1": 'viral protein',
"T1235-D1": 'viral protein',
"T1237-D1": 'bacteria protein',
"T1239-D1-all":'bacteria protein',
"T1239-D2-all":'bacteria protein',
"T1239-D3-all":'bacteria protein',
"T1239-D4-all":'bacteria protein',
"T1240-D1":'bacteria protein',
"T1240-D2":'bacteria protein',
"T1243-D1":'bacteria protein',
"T1244s1-D1":'protozoan parasites',
"T1245s1-D1":'fast evolving',
"T1245s2-D1":'fast evolving',
"T1246-D1":'bacteria protein',
"T1249v1-D1":'viral protein',
"T1257":'viral protein',
"T1259-D1":'bacteria protein',
"T1266-D1":'human protein',
"T1267s1-D1":'bacteria protein',
"T1267s1-D2":'bacteria protein',
"T1267s2-D1":'bacteria protein',
"T1269-D1":'human protein',
"T1269-D2":'human protein',
"T1269-D3":'human protein',
"T1270-D1":'bacteria protein',
"T1270-D2":'bacteria protein',
"T1271s1-D1":'protozoan parasite',
"T1271s2-D1":'protozoan parasite',
"T1271s3-D1":'protozoan parasite',
"T1271s4-D1":'protozoan parasite',
"T1271s5-D1":'protozoan parasite',
"T1271s5-D2":'protozoan parasite',
"T1271s6-D1":'protozoan parasite',
"T1271s7-D1":'protozoan parasite',
"T1271s8-D1":'protozoan parasite',
"T1271s8-D2":'protozoan parasite',
"T1272s2-D1":'mammal protein',
"T1272s6-D1":'mammal protein',
"T1272s8-D1":'mammal protein',
"T1272s9-D1":'mammal protein',
"T1274-D1":'bacteria protein',
"T1276-D1":'ants protein',
"T1278-D1":'bacteria protein',
"T1279-D1":'bacteria protein',
"T1279-D2":'bacteria protein',
"T1280-D1":'viral protein',
"T1284-D1":'viral protein',
"T1292-D1":'bacteria protein',
"T1294-D1-all":'mold protein',
"T1295-D1":'nanobody',
"T1295-D2":'nanobody',
"T1295-D3":'nanobody',
"T1298-D1":'bacteria protein',
"T1298-D2":'bacteria protein',
"T1299-D1":'Archaea protein',
}

len(column_information_1)

column_colors_1 = ["#4682B4", "#32CD32", "#FF4500", "#DA70D6", "#8A2BE2", "#FF6347", "#FF69B4", "#40E0D0", "#FFD700", "#ADFF2F",      "#FF7F50",  # 珊瑚色 (Coral)
    "#6495ED",  # 矢车菊蓝 (Cornflower Blue)
    "#DC143C",  # 猩红 (Crimson)
    "#00CED1",  # 深青色 (Dark Turquoise)
    "#FF1493",  # 深粉红 (Deep Pink)
    "#00FA9A",  # 中春绿色 (Medium Spring Green)
    "#BA55D3",  # 洋兰 (Medium Orchid)
    "#FFA500",  # 橙色 (Orange)
    "#7B68EE",  # 中暗蓝色 (Medium Slate Blue)
    "#20B2AA"   # 浅海蓝 (Light Sea Green)
    ]
    
col_info_to_color = {}
column_colors_list_1 = []
for key, value in column_information_1.items():
    if value not in col_info_to_color:
        col_info_to_color[value] = column_colors_1.pop()
    column_colors_list_1.append(col_info_to_color[value])

col_info_to_color
column_colors_list_1


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm

# Step 1: Calculate row sums
sum = heatmap_data.sum(axis=1)

# Step 2: Sort indices based on row sums
sorted_indices = sum.sort_values(ascending=False).index

# Step 3: Sort the heatmap data and row sums, reset index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)

# Step 4: Sort mask based on sorted indices
sorted_mask = pd.DataFrame(mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)

# Step 5: Mask the sorted heatmap data
masked_data = sorted_heatmap_data.where(~sorted_mask)

# Step 6: Fill NaN values in masked_data with 50 for distance calculation
filled_data = masked_data.fillna(50)

# Step 7: Create custom colormap
colors = ["blue", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
cmap.set_bad(color='gray')  # Set the masked area to gray

# Step 8: Create a diverging norm to set midpoint at 50
norm = TwoSlopeNorm(vmin=filled_data.min().min(), vmax=filled_data.max().max(), vcenter=50)

col_colors=[column_colors_list_1]
# Step 9: Plot heatmap using clustermap
plt.figure(dpi=300)  # 设置 DPI（例如，150 DPI）
g=sns.clustermap(
    figsize=(14, 20),
    data=filled_data,
    dendrogram_ratio=(0.1, 0.1),  # Reduce the space taken by dendrograms
    cmap=cmap,                  # Use the custom blue-white-red colormap
    norm=norm,                  # Use diverging norm to center at 50
    linewidths=0.5,             # Set the linewidth between cells
    linecolor='white',          # Set the line color
    # cbar_kws={'shrink': 0.3,'aspect': 10, 'pad': 0.02, 'label': 'Intensity'},  # Add label to the color bar
    xticklabels=filled_data.columns,  # Use original column names as labels
    yticklabels=sorted_indices,       # Use sorted indices as row labels
    col_colors=col_colors,
    mask=sorted_mask,            # Apply mask to the heatmap
    square=True, 
    method="ward",
)
g.fig.set_dpi(300)  # 再次设置为 150 DPI，如果需要更高分辨率，可以增加该值

# Show heatmap
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 定义每个注解类别的颜色映射
colors_1 = {'Group A': 'red', 'Group B': 'blue', 'Group C': 'green'}
colors_2 = {'Type X': 'purple', 'Type Y': 'orange'}
colors_3 = {'High': 'darkgreen', 'Medium': 'yellow', 'Low': 'lightgrey'}

# 创建每个类别的图例图
def plot_legend(colors, title):
    fig, ax = plt.subplots(figsize=(3, 1), dpi=300)
    legend_elements = [mpatches.Patch(color=color, label=label) for label, color in colors.items()]
    ax.legend(handles=legend_elements, loc='center', title=title, frameon=False)
    ax.axis('off')  # 隐藏坐标轴
    plt.show()

# 绘制每个类别的图例
plot_legend(colors_1, "Category 1")
plot_legend(colors_2, "Category 2")
plot_legend(colors_3, "Category 3")
plot_legend(col_info_to_color, "targets")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, TwoSlopeNorm
from matplotlib.gridspec import GridSpec

# Step 1: Calculate row sums
sum = heatmap_data.sum(axis=1)

# Step 2: Sort indices based on row sums
sorted_indices = sum.sort_values(ascending=False).index

# Step 3: Sort the heatmap data and row sums, reset index
sorted_heatmap_data = heatmap_data.loc[sorted_indices].reset_index(drop=True)
sorted_sum = sum.loc[sorted_indices].reset_index(drop=True)

# Step 4: Sort mask based on sorted indices
sorted_mask = pd.DataFrame(mask, index=heatmap_data.index).loc[sorted_indices].reset_index(drop=True)

# Step 5: Mask the sorted heatmap data
masked_data = sorted_heatmap_data.where(~sorted_mask)

# Step 6: Fill NaN values in masked_data with 50 for distance calculation
filled_data = masked_data.fillna(50)

# Step 7: Create custom colormap
colors = ["blue", "white", "red"]
cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=256)
cmap.set_bad(color='gray')  # Set the masked area to gray

# Step 8: Create a diverging norm to set midpoint at 50
norm = TwoSlopeNorm(vmin=filled_data.min().min(), vmax=filled_data.max().max(), vcenter=50)

col_colors = [column_colors_list_1]

# Step 9: Create figure with custom GridSpec
fig = plt.figure(figsize=(18, 25))
gs = GridSpec(2, 2, width_ratios=[0.05, 0.95], height_ratios=[0.05, 0.95], wspace=0.1, hspace=0.1)

# Step 10: Plot heatmap without color bar
ax = fig.add_subplot(gs[1, 1])
sns.heatmap(
    filled_data,
    ax=ax,
    cmap=cmap,                  # Use the custom blue-white-red colormap
    norm=norm,                  # Use diverging norm to center at 50
    linewidths=0.01,            # Set the linewidth between cells
    linecolor='white',          # Set the line color
    xticklabels=filled_data.columns,  # Use original column names as labels
    yticklabels=sorted_indices,       # Use sorted indices as row labels
    mask=sorted_mask,            # Apply mask to the heatmap
    square=True,
    cbar=False,                 # Disable color bar in the heatmap
)

# Step 11: Add color bar separately
cax = fig.add_subplot(gs[1, 0])
cbar = plt.colorbar(ax.collections[0], cax=cax, orientation='vertical')
cbar.set_label('Intensity')
cbar.ax.tick_params(labelsize=10)

# Show heatmap
plt.show()
